## Import statements and globals

In [5]:
#from pulp import *
from gurobi import *
import math
import pandas as pd
import numpy as np
from sklearn import preprocessing


HitterPositions = ['C', '1B', '2B', 'SS', '3B', 'OF', 'outer', 'inner', 'util']
HitterMetrics = ['R', 'HR', 'RBI', 'SB', 'AVG']

PitcherPositions = ['SP', 'RP']
PitcherMetrics = ['W', 'SV', 'K', 'ERA', 'WHIP']


R = sorted([1183,1145,1125,1100,1082,1064,1046,1028,1006,980,949,893])
HR = sorted([383,366,355,345,338,328,324,314,305,296,282,261])
RBI = sorted([1137,1104,1079,1058,1041,1023,1007,987,964,942,910,855])
SB = sorted([152,138,128,121,114,108,103,96,90,84,76,64])
AVG = sorted([0.278,0.274,0.272,0.270,0.269,0.267,0.266,0.264,0.262,0.260,0.258,0.254])

K = sorted([1622,1557,1509,1470,1438,1404,1370,1332,1290,1245,1182,1070])
W = sorted([103,97,94,91,88,85,83,80,77,74,69,61])
SV = sorted([100,89,82,77,71,66,61,54,48,40,31,18])
ERA = sorted([3.497,3.651,3.754,3.834,3.909,4.041,4.116,4.192,4.276,4.375,4.525],reverse=True)
WHIP = sorted([1.131,1.164,1.184,1.200,1.214,1.228,1.242,1.256,1.270,1.285,1.303,1.329],reverse=True)


hitter_scores = {'R':R,'HR':HR,'RBI':RBI,'SB':SB,'AVG':AVG}
pitcher_scores = {'K':K,'W':W,'SV':SV,'ERA':ERA,'WHIP':WHIP}

hitter_maxes = [max(hitter_scores['R']),max(hitter_scores['HR']),max(hitter_scores['RBI']),max(hitter_scores['SB']),max(hitter_scores['AVG'])]
pitcher_maxes = [max(pitcher_scores['W']),max(pitcher_scores['SV']),max(pitcher_scores['K']),min(pitcher_scores['ERA']),min(pitcher_scores['WHIP'])]

hitter_mins = [hitter_scores['R'][5],hitter_scores['HR'][5],hitter_scores['RBI'][5],hitter_scores['SB'][5],hitter_scores['AVG'][5]]
pitcher_mins = [pitcher_scores['W'][5],pitcher_scores['SV'][5],pitcher_scores['K'][5],pitcher_scores['ERA'][5],pitcher_scores['WHIP'][5]]

pitcher_avg_maxes
hitter_mins

ModuleNotFoundError: No module named 'gurobi'

## Import data

In [2]:
def get_hitters():
    hitters = pd.read_csv('razzball-hitters.csv', index_col='#', usecols=['#','Name','Team','ESPN','R','HR', 'RBI', 'SB','AVG','AB','H'])
    hitters.rename_axis('Razzball_Rank', inplace=True)
    hitters.reset_index(inplace=True)

    # sort and rank
    for metric in HitterMetrics:
        hitters.sort_values(by=[metric],inplace=True, ascending=False)
        hitters.reset_index(inplace=True, drop=True)
        hitters.index.rename('{} rank'.format(metric), inplace=True)
        hitters.reset_index(inplace=True)
    hitters['Ovr'] = (hitters['AVG rank'] + hitters['SB rank'] + hitters['RBI rank'] + hitters['HR rank'] + hitters['R rank']) / 5
    #hitters['Ovr'] = (hitters['Ovr'] + hitters['Razzball_Rank']) / 2
    hitters.rename(columns={'ESPN':'POS'}, inplace=True)
    
    hitters = hitters.assign(POS=hitters.POS.str.split('/')).explode('POS')
    hitters.sort_values(by=['Ovr'],inplace=True,ascending=True)
    return hitters

def get_starting_pitchers():
    pitchers = pd.read_csv('razzball-pitchers.csv', index_col='#', usecols=['#','Name','Team','POS','W', 'SV', 'K', 'ERA', 'WHIP','IP','BB','H', 'ER'])
    pitchers.rename_axis('Razzball_Rank', inplace=True)
    pitchers.reset_index(inplace=True)
    pitchers.rename(columns={'H':'Hits'}, inplace=True)
    
    pitchers = pitchers.assign(POS=pitchers.POS.str.split('/')).explode('POS')
    sp = pitchers[pitchers['POS'] == 'SP'].reset_index(drop=True)
    rp = pitchers[pitchers['POS'] == 'RP'].reset_index(drop=True)
    
    for metric in PitcherMetrics:
        if(metric != 'SV'):
            sp.sort_values(by=[metric],inplace=True, ascending=(metric=='WHIP' or metric=='ERA'))
            sp.reset_index(inplace=True, drop=True)
            sp.rename_axis('{} rank'.format(metric), inplace=True)
            sp.reset_index(inplace=True)
    
    sp['Ovr'] = (sp['W rank'] + sp['K rank'] + sp['ERA rank'] + sp['WHIP rank']) / 4
    sp.sort_values(by=['Ovr'],inplace=True,ascending=True)
    
    return sp

def get_closing_pitchers():
    pitchers = pd.read_csv('razzball-pitchers.csv', index_col='#', usecols=['#','Name','Team','POS','W', 'SV', 'K', 'ERA', 'WHIP','IP','BB','H', 'ER'])
    pitchers.rename_axis('Razzball_Rank', inplace=True)
    pitchers.reset_index(inplace=True)
    pitchers.rename(columns={'H':'Hits'}, inplace=True)
    
    pitchers = pitchers.assign(POS=pitchers.POS.str.split('/')).explode('POS')
    rp = pitchers[pitchers['POS'] == 'RP'].reset_index(drop=True)
    
    for metric in PitcherMetrics:
        if(metric != 'W'):
            rp.sort_values(by=[metric],inplace=True, ascending=(metric=='WHIP' or metric=='ERA'))
            rp.reset_index(inplace=True, drop=True)
            rp.rename_axis('{} rank'.format(metric), inplace=True)
            rp.reset_index(inplace=True)
    
    rp['Ovr'] = (rp['SV rank'] + rp['K rank'] + rp['ERA rank'] + rp['WHIP rank']) / 4
    rp.sort_values(by=['Ovr'],inplace=True,ascending=True)
    
    return rp
    
def get_hitter_prices(hitters):
    prices = pd.read_csv('razzball-hitters-prices.csv', index_col='#', usecols=['#', 'Name', 'Team', '5×5 $', '$R', '$HR', '$RBI', '$SB', '$AVG (no OBP)'])
    prices.rename(columns={'5×5 $': '$'},inplace=True)
    prices['$'] = prices['$'].apply(lambda x: 1 if x <=1 else x)
    hitters = hitters.merge(prices, left_on=['Name', 'Team'], right_on=['Name','Team'], how='left')
    return hitters

def get_pitcher_prices(pitchers):
    prices = pd.read_csv('razzball-pitchers-prices.csv', index_col='#', usecols=['#','Name','Team','5×5 $','$W (no QS)','$SV (no HLD)','$K','$WHIP','$ERA'])
    prices.rename(columns={'5×5 $': '$'},inplace=True)
    prices['$'] = prices['$'].apply(lambda x: 1 if x <=1 else x)
    pitchers = pitchers.merge(prices, left_on=['Name', 'Team'], right_on=['Name','Team'], how='left')
    return pitchers
    
def split_by_pos(hitters, pos):
    if (pos == 'C'):
        return hitters[hitters['POS'] == 'C']
    elif (pos == '1B'):
        return hitters[hitters['POS'] == '1B']
    elif (pos == '2B'):
        return hitters[hitters['POS'] == '2B']
    elif (pos == 'SS'):
        return hitters[hitters['POS'] == 'SS']
    elif (pos == '3B'):
        return hitters[hitters['POS'] == '3B']
    elif (pos == 'OF'):
        return hitters[hitters['POS'] == 'OF']
    elif (pos == 'corner'):
        return hitters[hitters['POS'] == '1B'].append(hitters[hitters['POS'] == '3B'])
    elif (pos == 'inner'):
        return hitters[hitters['POS'] == '2B'].append(hitters[hitters['POS'] == 'SS'])
    elif (pos == 'SP'):
        return hitters[hitters['POS'] == 'SP']
    elif (pos == 'RP'):
        return hitters[hitters['POS'] == 'RP']
    else: # util
        return hitters
        

## Run calculations

In [3]:
def group_players(players):
    bandwidth = estimate_bandwidth(players, quantile=0.05, n_samples=100)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    labels = ms.fit_predict(players)
    #labels = ms.labels_
    cluster_centers = ms.cluster_centers_

    #labels_unique = np.unique(labels)
    #n_clusters_ = len(labels_unique)

    #print("number of estimated clusters : %d" % n_clusters_)
    players['group'] = labels
    return players

In [4]:
def group_pitchers_pos(pitchers):
    return [group_players(split_by_pos(pitchers, x).drop(['W rank', 'SV rank', 'K rank', 'ERA rank', 'WHIP rank', 'Razzball_Rank','Team','POS'], axis=1)) for x in PitcherPositions]

def group_by_pos(hitters):
    return [group_players(split_by_pos(hitters, x).drop(['AVG rank', 'SB rank', 'RBI rank', 'HR rank', 'R rank', 'Razzball_Rank','Team','POS'], axis=1)) for x in HitterPositions]

### Prep dataframes

In [5]:
hitter_tables = { 'C': "",
                  '1B': "",
                  '2B': "",
                  'SS': "",
                  '3B': "",
                  'inner': "",
                  'outer': "",
                  'OF': "",
                  'util': "" }

pitcher_tables = { 'SP': "",
                   'RP': "" }

In [6]:
hitters = get_hitters().set_index('Name')
groups = []
#C, 1B, 2B, SS, 3B, OF, outer, inner, util
#groups = group_by_pos(hitters)
groups = [split_by_pos(hitters, x) for x in HitterPositions]
#hitters['group'] = group_players(hitters.drop(['AVG rank', 'SB rank', 'RBI rank', 'HR rank', 'R rank', 'Razzball_Rank','Team','ESPN'], axis=1))
for i in range(len(groups)):
    groups[i] = groups[i].sort_values(by=['Ovr'],ascending=True)
    hitter_tables[HitterPositions[i]] = groups[i]
    #groups[i].to_csv('{}.csv'.format(HitterPositions[i]))

pitchers = get_pitchers().set_index('Name')
#pitchers
pitchers.sort_values(by=['Razzball_Rank'],inplace=True,ascending=True)
pitchers
#groups = group_pitchers_pos(pitchers)
#groups = [split_by_pos(pitchers, x) for x in PitcherPositions]
#for i in range (len(groups)):
#    groups[i] = groups[i].sort_values(by=['Ovr'],ascending=True)
#    pitcher_tables[PitcherPositions[i]] = groups[i]
    
#pitcher_tables['RP']


NameError: name 'get_pitchers' is not defined

In [16]:
def find_min():
    # 20 1B, 20 2B, 20 SS, 20 3B
    # 70 OF, 100 SP, 45 RP
    # = 50,400,000,000 iterations
    
    # 18 1B, 18 2B, 18 SS, 18 3B
    # 70 OF, 80 SP, 40 RP
    # = 23,514,624,000 iterations
    prob = LpProblem("IdealRosterProblem",LpMinimize)
    
    all_players = get_all_players()
    
    all_names = list(all_players.index)
    name_list = list(all_players['Name'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    allRuns = dict(zip(all_names,all_players['R rank']))
    allHRs = dict(zip(all_names,all_players['HR rank']))
    allRBIs = dict(zip(all_names,all_players['RBI rank']))
    allSBs = dict(zip(all_names,all_players['SB rank']))
    allAVG = dict(zip(all_names,all_players['AVG rank']))
    
    allWs = dict(zip(all_names,all_players['W rank']))
    allKs = dict(zip(all_names,all_players['K rank']))
    allSVs = dict(zip(all_names,all_players['SV rank']))
    allERA = dict(zip(all_names,all_players['ERA rank']))
    allWHIP = dict(zip(all_names,all_players['WHIP rank']))
    allPOS = dict(zip(all_names,all_players['POS']))
    
    player_vars = LpVariable.dicts("Player",all_names,lowBound=0,upBound=1,cat='Integer')
    player_chosen = LpVariable.dicts("Chosen",name_list,0,1,cat='Integer')
    
    print("all plrs: ", len(all_players.index))
    print('vars: ', len(player_vars))
    print('nms: ', len(all_names))
    
    prob += lpSum([(allRuns[i]+allHRs[i]+allRBIs[i]+allSBs[i]+allAVG[i]+allWs[i]+allKs[i]+allSVs[i]+allERA[i]+allWHIP[i])*player_vars[i] for i in all_names])
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 260
    
    prob += lpSum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1
    
    # update these based on position depth
    prob += lpSum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 1
    prob += lpSum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2
    
    prob += lpSum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6
    
    prob += lpSum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8
    
    prob += lpSum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3
    
    prob += lpSum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0
    
    
    # ensure no player is selected twice
    #sum(isEntryUsed[x] for x in range(len(entries)) if entries[x].name == name) <= 1
    #prob += lpSum([all_players.iloc[int(player_vars[i].name.split('_')[1])] for i in all_names]) == 1
    #prob += lpSum([all_players.iloc[i] for i in all_names if player_vars[i] == 1]) == 25
    for f in all_names:
        prob += player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.1
        prob += player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e5
    
    prob += lpSum(player_chosen)==25
    
    
    prob.solve()
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    #print(prob.variables())
    for v in prob.variables():
        if v.varValue>0 and not 'Chosen' in v.name:
            #print(v.name, "=", v.varValue)
            #print(all_players.index)
            #print(v.name.split('_')[1])
            #print(all_players.loc[int(v.name.split('_')[1])])
            Team = Team.append(all_players.loc[int(v.name.split('_')[1])])
            
            #print(all_players.loc[nm[1] + " " + nm[2]])
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)
            

def find_max():
    prob = LpProblem("IdealRosterProblem",LpMaximize)
    
    #all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'), ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(30,'Ovr'), ignore_index=True)
    #all_players.rename_axis('idx', inplace=True)
    
    #all_players.fillna(0, inplace=True)
    
    #norm_players = all_players[['R', 'HR', 'RBI', 'SB', 'AVG', 'W', 'SV', 'K', 'ERA', 'WHIP']]
    #norm_players = pd.DataFrame(preprocessing.normalize(norm_players),columns=['R', 'HR', 'RBI', 'SB', 'AVG', 'W', 'SV', 'K', 'ERA', 'WHIP'])
    
    
    #return(all_players[['normR', 'normHR', 'normRBI', 'normSB', 'normAVG', 'normW', 'normSV', 'normK', 'normERA', 'normWHIP']])
    all_players = get_all_players()
    
    
    all_names = list(all_players.index)
    name_list = list(all_players['Name'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    #total_R = sum(all_players['R'])
    #total_HR = sum(all_players['HR'])
    #total_RBI = sum(all_players['RBI'])
    #total_SB = sum(all_players['SB'])
    #total_H = sum(all_players['H'])
    #total_AB = sum(all_players['AB'])
    
    total_R = np.mean(all_players['R'])
    total_HR = np.mean(all_players['HR'])
    total_RBI = np.mean(all_players['RBI'])
    total_SB = np.mean(all_players['SB'])
    total_H = np.mean(all_players['H'])
    total_AB = np.mean(all_players['AB'])
    
    
    #all_players['AdjAVG'] = all_players['AVG'].apply(lambda x: 0 if x == 0 else 1/x)
    
    allRuns = dict(zip(all_names,all_players['normR']))
    allHRs = dict(zip(all_names,all_players['normHR']))
    allRBIs = dict(zip(all_names,all_players['normRBI']))
    allSBs = dict(zip(all_names,all_players['normSB']))
    allAVG = dict(zip(all_names,all_players['normAVG']))
    #allH = dict(zip(all_names,all_players['normH']))
    #allAB = dict(zip(all_names,all_players['normAB']))
    #allAdjAVG = dict(zip(all_names,all_players['AVG']/max(all_players['AVG'])))
    
    #total_W = sum(all_players['W'])
    #total_K = sum(all_players['K'])
    #total_SV = sum(all_players['SV'])
    #total_IP = sum(all_players['IP'])
    #total_ER = sum(all_players['ER'])
    #total_BB = sum(all_players['BB'])
    #total_Hits = sum(all_players['Hits'])
    
    total_W = np.mean(all_players['W'])
    total_K = np.mean(all_players['K'])
    total_SV = np.mean(all_players['SV'])
    total_IP = np.mean(all_players['IP'])
    total_ER = np.mean(all_players['ER'])
    total_BB = np.mean(all_players['BB'])
    total_Hits = np.mean(all_players['Hits'])
    
    #all_players['AdjER'] = 0 if all_players['ER'] == 0 else 1/all_players['ER']
    all_players['AdjER'] = all_players['ER'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjBB'] = all_players['BB'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjHits'] = all_players['Hits'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjERA'] = all_players['normERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjWHIP'] = all_players['normWHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    
    allWs = dict(zip(all_names,all_players['normW']))
    allKs = dict(zip(all_names,all_players['normK']))
    allSVs = dict(zip(all_names,all_players['normSV']))
    allERA = dict(zip(all_names,all_players['normERA']))
    allWHIP = dict(zip(all_names,all_players['normWHIP']))
    allPOS = dict(zip(all_names,all_players['POS']))
    allIP = dict(zip(all_names,all_players['IP']/total_IP))
    allER = dict(zip(all_names,all_players['AdjER']))
    allBB = dict(zip(all_names,all_players['AdjBB']))
    allHits = dict(zip(all_names,all_players['AdjHits']))
    allAdjERA = dict(zip(all_names,all_players['AdjERA']))
    allAdjWHIP = dict(zip(all_names,all_players['AdjWHIP']))
    
    
    player_vars = LpVariable.dicts("Player",all_names,lowBound=0,upBound=1,cat='Integer')
    player_chosen = LpVariable.dicts("Chosen",name_list,0,1,cat='Integer')
    
    print("all plrs: ", len(all_players.index))
    print('vars: ', len(player_vars))
    print('nms: ', len(all_names))
    
    prob += lpSum([allRuns[i]*player_vars[i] for i in all_names])
    prob += lpSum([allHRs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allRBIs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSBs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAVG[i]*player_vars[i] for i in all_names])
    prob += lpSum([allWs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allKs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSVs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAdjERA[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAdjWHIP[i]*player_vars[i] for i in all_names])
    
    #prob += lpSum(score({'R':[allRuns[i]*player_vars[i] for i in all_names]}))
    
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 260
    
    prob += lpSum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1
    
    # update these based on position depth
    prob += lpSum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 1
    prob += lpSum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2
    
    prob += lpSum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6
    
    prob += lpSum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8
    
    prob += lpSum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3
    prob += lpSum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0
    
    
    # ensure no player is selected twice
    #sum(isEntryUsed[x] for x in range(len(entries)) if entries[x].name == name) <= 1
    #prob += lpSum([all_players.iloc[int(player_vars[i].name.split('_')[1])] for i in all_names]) == 1
    #prob += lpSum([all_players.iloc[i] for i in all_names if player_vars[i] == 1]) == 25
    for f in all_names:
        prob += player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.1
        prob += player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e5
    
    prob += lpSum(player_chosen)==25

    LpSolverDefault.msg = 1 
    prob.solve()
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        if v.varValue>0 and not 'Chosen' in v.name:
            #print(v.name, "=", v.varValue)
            #print(all_players.index)
            #print(v.name.split('_')[1])
            #print(all_players.loc[int(v.name.split('_')[1])])
            Team = Team.append(all_players.loc[int(v.name.split('_')[1])])
            
            #print(all_players.loc[nm[1] + " " + nm[2]])
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)
 

def find_both():
    prob = LpProblem("IdealRosterProblem",LpMaximize)
    
    all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'), ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(30,'Ovr'), ignore_index=True)
    all_players.rename_axis('idx', inplace=True)
    
    all_players.fillna(0, inplace=True)
    
    all_names = list(all_players.index)
    name_list = list(all_players['Name'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    total_R = sum(all_players['R'])
    total_HR = sum(all_players['HR'])
    total_RBI = sum(all_players['RBI'])
    total_SB = sum(all_players['SB'])
    total_H = sum(all_players['H'])
    total_AB = sum(all_players['AB'])
    #all_players['AdjAVG'] = all_players['AVG'].apply(lambda x: 0 if x == 0 else 1/x)
    
    allRuns = dict(zip(all_names,all_players['R']/total_R))
    allHRs = dict(zip(all_names,all_players['HR']/total_HR))
    allRBIs = dict(zip(all_names,all_players['RBI']/total_RBI))
    allSBs = dict(zip(all_names,all_players['SB']/total_SB))
    allAVG = dict(zip(all_names,all_players['AVG']))
    allH = dict(zip(all_names,all_players['H']/total_H))
    allAB = dict(zip(all_names,all_players['AB']/total_AB))
    allAdjAVG = dict(zip(all_names,all_players['AVG']/max(all_players['AVG'])))
    
    total_W = sum(all_players['W'])
    total_K = sum(all_players['K'])
    total_SV = sum(all_players['SV'])
    total_IP = sum(all_players['IP'])
    total_ER = sum(all_players['ER'])
    total_BB = sum(all_players['BB'])
    total_Hits = sum(all_players['Hits'])
    
    #all_players['AdjER'] = 0 if all_players['ER'] == 0 else 1/all_players['ER']
    all_players['AdjER'] = all_players['ER'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjBB'] = all_players['BB'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjHits'] = all_players['Hits'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjERA'] = all_players['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjWHIP'] = all_players['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    
    allWs = dict(zip(all_names,all_players['W']/total_W))
    allKs = dict(zip(all_names,all_players['K']/total_K))
    allSVs = dict(zip(all_names,all_players['SV']/total_SV))
    allERA = dict(zip(all_names,all_players['ERA']))
    allWHIP = dict(zip(all_names,all_players['WHIP']))
    allPOS = dict(zip(all_names,all_players['POS']))
    allIP = dict(zip(all_names,all_players['IP']/total_IP))
    allER = dict(zip(all_names,all_players['AdjER']))
    allBB = dict(zip(all_names,all_players['AdjBB']))
    allHits = dict(zip(all_names,all_players['AdjHits']))
    allAdjERA = dict(zip(all_names,all_players['AdjERA']))
    allAdjWHIP = dict(zip(all_names,all_players['AdjWHIP']))
    
    allRunsRank = dict(zip(all_names,all_players['R rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allHRsRank = dict(zip(all_names,all_players['HR rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allRBIsRank = dict(zip(all_names,all_players['RBI rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allSBsRank = dict(zip(all_names,all_players['SB rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allAVGRank = dict(zip(all_names,all_players['AVG rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    
    allWsRank = dict(zip(all_names,all_players['W rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allKsRank = dict(zip(all_names,all_players['K rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allSVsRank = dict(zip(all_names,all_players['SV rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allERARank = dict(zip(all_names,all_players['ERA rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    allWHIPRank = dict(zip(all_names,all_players['WHIP rank'].apply(lambda x: 1 if x == 0 else 1/x)))
    
    player_vars = LpVariable.dicts("Player",all_names,lowBound=0,upBound=1,cat='Integer')
    player_chosen = LpVariable.dicts("Chosen",name_list,0,1,cat='Integer')
    
    print("all plrs: ", len(all_players.index))
    print('vars: ', len(player_vars))
    print('nms: ', len(all_names))
    
    prob += lpSum([(allRuns[i]+allHRs[i]+allRBIs[i]+allSBs[i]+allAdjAVG[i]+allWs[i]+allKs[i]+allSVs[i]+allAdjERA[i]+allAdjWHIP[i])*player_vars[i] for i in all_names])
    
    prob += lpSum([(allRunsRank[i]+allSBsRank[i]+allAVGRank[i]+allERARank[i]+allWHIPRank[i])*player_vars[i] for i in all_names])

    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 260
    
    prob += lpSum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1
    
    # update these based on position depth
    prob += lpSum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 1
    prob += lpSum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2
    
    prob += lpSum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6
    
    prob += lpSum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8
    
    prob += lpSum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3
    prob += lpSum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0
    
    
    # ensure no player is selected twice
    #sum(isEntryUsed[x] for x in range(len(entries)) if entries[x].name == name) <= 1
    #prob += lpSum([all_players.iloc[int(player_vars[i].name.split('_')[1])] for i in all_names]) == 1
    #prob += lpSum([all_players.iloc[i] for i in all_names if player_vars[i] == 1]) == 25
    for f in all_names:
        prob += player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.1
        prob += player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e5
    
    prob += lpSum(player_chosen)==25
    
    LpSolverDefault.msg = 1 
    prob.solve()
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        if 'dummy' not in v.name and v.varValue>0 and not 'Chosen' in v.name:
            #print(v.name, "=", v.varValue)
            #print(all_players.index)
            #print(v.name.split('_')[1])
            #print(all_players.loc[int(v.name.split('_')[1])])
            Team = Team.append(all_players.loc[int(v.name.split('_')[1])])
            
            #print(all_players.loc[nm[1] + " " + nm[2]])
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)
 
Team_max = find_max()
#Team_max[['Name','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]
#print('-- Name --|-- Pos --|-- Avg --|-- R --|-- RBI --|-- HR --|-- SB --|-- WHIP --|-- ERA --|-- W --|-- SV --|-- K --|--$--')
#for i in range(len(Team_max)):
#    print(Team_max.iloc[i]['Name'],Team_max.iloc[i]['POS'],Team_max.iloc[i]['AVG'],Team_max.iloc[i]['R'],Team_max.iloc[i]['RBI'],Team_max.iloc[i]['HR'],Team_max.iloc[i]['SB'],Team_max.iloc[i]['WHIP'],Team_max.iloc[i]['ERA'],Team_max.iloc[i]['W'],Team_max.iloc[i]['SV'],Team_max.iloc[i]['K'],Team_max.iloc[i]['$'])
    
#print('Total','--   --',sum(Team_max['H'])/sum(Team_max['AB']),sum(Team_max['R']),sum(Team_max['RBI']),sum_max(Team['HR']),sum(Team_max['SB']),(sum(Team_max['BB'])+sum(Team_max['Hits']))/sum(Team_max['IP']),(sum(Team_max['ER'])/sum(Team_max['IP']))*9,sum(Team_max['W']),sum(Team_max['SV']),sum(Team_max['K']),sum(Team_max['$']))
#print(Team_max['POS'].describe())
#Team_max[['Name','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]

Team_min = find_min()
#print(Team_max['POS'].describe())
#Team_min[['Name','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]
#print(Team.columns)
#print('-- Name --|-- Pos --|-- Avg --|-- R --|-- RBI --|-- HR --|-- SB --|-- WHIP --|-- ERA --|-- W --|-- SV --|-- K --|--$--')
#for i in range(len(Team_min)):
#    print(Team_min.iloc[i]['Name'],Team_min.iloc[i]['POS'],Team_min.iloc[i]['AVG'],Team_min.iloc[i]['R'],Team_min.iloc[i]['RBI'],Team_min.iloc[i]['HR'],Team_min.iloc[i]['SB'],Team_min.iloc[i]['WHIP'],Team_min.iloc[i]['ERA'],Team_min.iloc[i]['W'],Team_min.iloc[i]['SV'],Team_min.iloc[i]['K'],Team_min.iloc[i]['$'])
    
#print('Total','--   --',sum(Team_min['H'])/sum(Team_min['AB']),sum(Team_min['R']),sum(Team_min['RBI']),sum(Team_min['HR']),sum(Team_min['SB']),(sum(Team_min['BB'])+sum(Team_min['Hits']))/sum(Team_min['IP']),(sum(Team_min['ER'])/sum(Team_min['IP']))*9,sum(Team_min['W']),sum(Team_min['SV']),sum(Team_min['K']),sum(Team_min['$']))
Max_Tot = Team_max[Team_max['Name'] == 'Total']
Min_Tot = Team_min[Team_min['Name'] == 'Total']
#Min_Tot
TeamMaxMin = Max_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]-Min_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]
#print(Max_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']])
#print(Min_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']])
Avg = find_both()
Avg_Tot = Avg[Avg['Name'] == 'Total']
TeamAvgMin = Avg_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']] - Min_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]

TeamAvgMax = Avg_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']] - Max_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]

#Tot = Max_Tot.append(Min_Tot).append(Avg_Tot)
#Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP']]

print('Max',get_score(Max_Tot.loc[25]))
print('Min',get_score(Min_Tot.loc[25]))
print('Avg',get_score(Avg_Tot.loc[25]))
#Min_Tot[['R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']]
Team_max[['Name','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']]




/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1380: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


all plrs:  300
vars:  300
nms:  300
Status: Optimal


/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


all plrs:  300
vars:  300
nms:  300
Status: Optimal
all plrs:  300
vars:  300
nms:  300


/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1380: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Status: Optimal
R
1 1029.8 898
2 1029.8 966
3 1029.8 974
4 1029.8 997
5 1029.8 1006
6 1029.8 1067
HR
1 300.79999999999995 260
2 300.79999999999995 284
3 300.79999999999995 291
4 300.79999999999995 302
RBI
1 1002.8000000000001 897
2 1002.8000000000001 905
3 1002.8000000000001 955
4 1002.8000000000001 1000
5 1002.8000000000001 1016
SB
1 94.29999999999998 72
2 94.29999999999998 73
3 94.29999999999998 94
4 94.29999999999998 97
AVG
1 0.26232318902700386 0.2592
2 0.26232318902700386 0.2601
3 0.26232318902700386 0.262
4 0.26232318902700386 0.2641
W
1 96.10000000000001 64
2 96.10000000000001 74
3 96.10000000000001 80
4 96.10000000000001 84
5 96.10000000000001 85
6 96.10000000000001 93
7 96.10000000000001 97
SV
1 63.0 3
2 63.0 39
3 63.0 42
4 63.0 54
5 63.0 55
6 63.0 59
7 63.0 59
8 63.0 73
K
1 1799.5 1132
2 1799.5 1330
3 1799.5 1336
4 1799.5 1387
5 1799.5 1391
6 1799.5 1480
7 1799.5 1531
8 1799.5 1598
9 1799.5 1643
10 1799.5 1725
11 1799.5 1788
ERA
1 3.659458443023693 4.616
2 3.659458443023693 4

,Name,R,HR,RBI,SB,AVG,W,SV,K,ERA,WHIP,Ovr,$
0,Eugenio Suarez,92.4,33.4,88.8,3.2,0.252000,0.0,0.0,0.0,0.000000,0.000000,127.60,12.1
1,Andrew McCutchen,80.0,23.3,63.6,6.8,0.259000,0.0,0.0,0.0,0.000000,0.000000,130.40,4.9
2,Didi Gregorius,70.8,22.7,81.5,5.2,0.259000,0.0,0.0,0.0,0.000000,0.000000,132.40,5.1
3,Avisail Garcia,60.0,19.9,69.5,6.2,0.269000,0.0,0.0,0.0,0.000000,0.000000,137.60,2.4
4,Jason Heyward,71.9,15.8,72.0,6.5,0.261000,0.0,0.0,0.0,0.000000,0.000000,144.00,2.2
5,Cesar Hernandez,67.8,10.1,54.5,9.1,0.278000,0.0,0.0,0.0,0.000000,0.000000,146.40,1.0
6,Shin-Soo Choo,83.4,21.0,61.7,9.0,0.249000,0.0,0.0,0.0,0.000000,0.000000,149.20,3.5
7,Corey Dickerson,64.6,18.3,65.9,4.1,0.271000,0.0,0.0,0.0,0.000000,0.000000,149.80,1.0
8,Jonathan Schoop,67.3,26.5,76.9,2.3,0.262000,0.0,0.0,0.0,0.000000,0.000000,155.20,3.8
9,Willy Adames,67.5,17.7,63.1,6.8,0.256000,0.0,0.0,0.0,0.000000,0.000000,159.20,1.0


In [8]:
all_players = get_hitters().set_index('Name').nsmallest(200,'Ovr').append(get_pitchers().set_index('Name').nsmallest(180,'Ovr'))
all_players.sort_values(by=['Ovr'], inplace=True)
all_players.drop(['AVG','R','RBI','HR','SB','WHIP','ERA','W','SV','K'], axis=1, inplace=True)
all_players

NameError: name 'get_pitchers' is not defined

In [244]:
def get_score(tm):
    score = {'R': 0,'HR':0,'RBI':0,'SB':0,'AVG':0,'K':0,'W':0,'SV':0,'ERA':0,'WHIP':0,'Total':0}
    R = sorted([1174,1136,1067,1006,1241,1110,974,997,1159,966,898])
    HR = sorted([433,352,353,284,382,321,291,332,355,302,260])
    RBI = sorted([1198,1088,1077,1030,1147,1016,955,1000,1075,905,897])
    SB = sorted([113,141,97,106,94,110,127,121,123,73,72])
    AVG = sorted([0.2735,0.2592,0.2740,0.2642,0.2768,0.2710,0.2620,0.2601,0.2705,0.2645,0.2641])
    K = sorted([1643,1531,1788,1598,1330,1387,1480,1725,1132,1391,1336])
    W = sorted([97,98,109,112,80,93,85,99,64,84,74])
    SV = sorted([59,73,3,55,115,79,105,42,39,59,54])
    ERA = sorted([3.907, 3.898,4.144,3.665,4.444,4.107,3.760,4.217,3.493,4.112,4.616],reverse=True)
    WHIP = sorted([1.216,1.244,1.262,1.102,1.339,1.247,1.210,1.291,1.131,1.267,1.284],reverse=True)
    
    hitter_scores = {'R':R,'HR':HR,'RBI':RBI,'SB':SB,'AVG':AVG}
    pitcher_scores = {'K':K,'W':W,'SV':SV,'ERA':ERA,'WHIP':WHIP}
    
    for metric in HitterMetrics:
        for s in range(len(hitter_scores[metric])):
            if(tm[metric] < hitter_scores[metric][s]):
                score[metric] = s+1
                score['Total'] += s+1
                break
        if (score[metric] == 0):
            score[metric] = 12
            score['Total'] += 12
    
    for metric in PitcherMetrics:
        if (metric != 'ERA') and (metric != 'WHIP'):
            for s in range(len(pitcher_scores[metric])):
                if(tm[metric] < pitcher_scores[metric][s]):
                    score[metric] = s+1
                    score['Total'] += s+1
                    break
        else:
            for s in range(len(pitcher_scores[metric])):
                if(tm[metric] > pitcher_scores[metric][s]):
                    score[metric] = s+1
                    score['Total'] += s+1
                    break
        if (score[metric] == 0):
            score[metric] = 12
            score['Total'] += 12
            
    print(score)
    return score

In [ ]:
get_all_players()[['normR']]

In [242]:
def score(tm,metric):
    R = sorted([1183,1145,1125,1100,1082,1064,1046,1028,1006,980,949,893])
    HR = sorted([383,366,355,345,338,328,324,314,305,296,282,261])
    RBI = sorted([1137,1104,1079,1058,1041,1023,1007,987,964,942,910,855])
    SB = sorted([152,138,128,121,114,108,103,96,90,84,76,64])
    AVG = sorted([0.278,0.274,0.272,0.270,0.269,0.267,0.266,0.264,0.262,0.260,0.258,0.254])
    
    K = sorted([1622,1557,1509,1470,1438,1404,1370,1332,1290,1245,1182,1070])
    W = sorted([103,97,94,91,88,85,83,80,77,74,69,61])
    SV = sorted([100,89,82,77,71,66,61,54,48,40,31,18])
    ERA = sorted([3.497,3.651,3.754,3.834,3.909,4.041,4.116,4.192,4.276,4.375,4.525],reverse=True)
    WHIP = sorted([1.131,1.164,1.184,1.200,1.214,1.228,1.242,1.256,1.270,1.285,1.303,1.329],reverse=True)
    
    hitter_scores = {'R':R,'HR':HR,'RBI':RBI,'SB':SB,'AVG':AVG}
    pitcher_scores = {'K':K,'W':W,'SV':SV,'ERA':ERA,'WHIP':WHIP}
    tm_sc = sum(tm)
    if metric in HitterMetrics:
        for s in range(len(hitter_scores[metric])):
            if(tm_sc < hitter_scores[metric][s]):
                score += s+1
                break
            if(s == len(hitter_scores[metric])):
                score += 12
    
    elif metric in PitcherMetrics:
        if (metric != 'ERA') and (metric != 'WHIP'):
            for s in range(len(pitcher_scores[metric])):
                if(tm_sc < pitcher_scores[metric][s]):
                    score += s+1
                    break
                if(s == len(pitcher_scores[metric])):
                    score += 12
        else:
            for s in range(len(pitcher_scores[metric])):
                if(tm_sc > pitcher_scores[metric][s]):
                    score += s+1
                    break
                if(s == len(pitcher_scores[metric])):
                    score += 12
    print(score)
    return score 

In [116]:
def get_all_players():
    scaler = preprocessing.StandardScaler(with_std=False)
    
    
    
    hitter_avg_maxes = [(hitter_maxes[i]/14) for i in range(len(hitter_maxes)-1)]
    hitter_avg_maxes.append(hitter_maxes[-1])
    pitcher_avg_maxes = [pitcher_maxes[i]/11 for i in range(len(pitcher_maxes)-2)]
    pitcher_avg_maxes.append(1/pitcher_maxes[-2])
    pitcher_avg_maxes.append(1/pitcher_maxes[-1])
    
    
    hitter_avg_mins = [(hitter_mins[i]/14) for i in range(len(hitter_mins)-1)]
    hitter_avg_mins.append(hitter_mins[-1])
    pitcher_avg_mins = [pitcher_mins[i]/11 for i in range(len(pitcher_mins)-2)]
    pitcher_avg_mins.append(1/pitcher_mins[-2])
    pitcher_avg_mins.append(1/pitcher_mins[-1])
    #print(pitcher_avg_mins)
    
    
    hitters = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr')
    norm_hitters = hitters[['R', 'HR', 'RBI', 'SB', 'AVG']]
    norm_hitters = pd.DataFrame(scaler.fit_transform(norm_hitters),columns=['R', 'HR', 'RBI', 'SB', 'AVG'])
    minH = math.inf
    maxH = 0
    for h in HitterMetrics:
        hitters['norm{}'.format(h)] = norm_hitters[h]
        if(max(norm_hitters[h])>maxH):
            maxH = max(norm_hitters[h])
        if(min(norm_hitters[h]<minH)):
            minH = min(norm_hitters[h])
    avg_max_ary = np.array(hitter_avg_maxes).reshape(1,-1)
    avg_min_ary = np.array(hitter_avg_mins).reshape(1,-1)
    hitter_avg_maxes = scaler.transform(avg_max_ary)
    hitterer_avg_mins = scaler.transform(avg_min_ary)
    
    
    scaler2 = preprocessing.MinMaxScaler((minH,maxH))
    starters = get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr')
    starters['SV']=np.NaN
    norm_starters = starters[['W']]
    norm_starters = pd.DataFrame(scaler.fit_transform(norm_starters),columns=['W'])
    starters['normW'] = norm_starters['W']
    
    win_max_ary = np.array(pitcher_avg_maxes[0]).reshape(1,-1)
    win_min_ary = np.array(pitcher_avg_mins[0]).reshape(1,-1)
    pitcher_temp = scaler.transform(win_max_ary)
    min_temp = scaler.transform(win_min_ary)
    #print(pitcher_temp)
        
    closers = get_pitcher_prices(get_closing_pitchers()).nsmallest(30,'SV rank')
    closers['W']=np.NaN
    norm_closers = closers[['SV']]
    norm_closers = pd.DataFrame(scaler.fit_transform(norm_closers),columns=['SV'])
    closers['normSV'] = norm_closers['SV']
    
    sv_max_ary = np.array(pitcher_avg_maxes[1]).reshape(1,-1)
    sv_min_ary = np.array(pitcher_avg_mins[1]).reshape(1,-1)
    pitcher_temp = np.append(pitcher_temp, scaler.transform(sv_max_ary))
    min_temp = np.append(min_temp,scaler.transform(sv_min_ary))
    #print(pitcher_temp)
        
    
    all_pitchers = starters.append(closers, ignore_index=True)
    all_pitchers['ERA'] = all_pitchers['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_pitchers['WHIP'] = all_pitchers['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    
    norm_pitchers = all_pitchers[['K', 'ERA', 'WHIP']]
    norm_pitchers = pd.DataFrame(scaler.fit_transform(norm_pitchers),columns=['K', 'ERA', 'WHIP'])
    for p in ['K', 'ERA', 'WHIP']:
        all_pitchers['norm{}'.format(p)] = norm_pitchers[p]
        
    pitcher_max_ary = np.array(pitcher_avg_maxes[2:5]).reshape(1,-1)
    pitcher_min_ary = np.array(pitcher_avg_mins[2:5]).reshape(1,-1)
    pitcher_temp = np.append(pitcher_temp,scaler.transform(pitcher_max_ary))
    min_temp = np.append(min_temp, scaler.transform(pitcher_min_ary))
    pitcher_avg_maxes = pitcher_temp
    pitcher_avg_mins = min_temp
    
    all_players = hitters.append(all_pitchers, ignore_index=True)
    all_players.rename_axis('idx', inplace=True)
    
    all_players['normW'].fillna(minH, inplace=True)
    all_players['normSV'].fillna(minH, inplace=True)
    all_players.fillna(0, inplace=True)
    return all_players,hitter_avg_maxes[0],pitcher_avg_maxes,hitter_avg_mins,pitcher_avg_mins

    
    
get_all_players()

print(get_all_players()[3])
print(get_all_players()[4])

/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


[74.71428571428571, 23.142857142857142, 71.92857142857143, 7.357142857142857, 0.266]
[-1.61287879e+00 -1.66212121e+01 -1.01752121e+01 -2.15387606e-03
  6.33362051e-03]


/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [226]:
def find_max_three():
    prob = LpProblem("IdealRosterProblem",LpMaximize)
    scaler = preprocessing.StandardScaler(with_std=False)
    
    #all_players,hitter_avg_maxes,pitcher_avg_maxes,hitter_avg_mins,pitcher_avg_mins = get_all_players()
    all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'),ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(30,'SV rank'),ignore_index=True)
    all_players.fillna(0, inplace=True)
    all_players['ERA'] = all_players['ERA'].apply(lambda x: 0 if x == 0 else 2/x)
    all_players['WHIP'] = all_players['WHIP'].apply(lambda x: 0 if x == 0 else 2/x)
    norm_players = all_players[['R', 'HR', 'RBI', 'SB', 'AVG','W','SV','K','ERA','WHIP']]
    
    norm_players = pd.DataFrame(scaler.fit_transform(norm_players),columns=['R', 'HR', 'RBI', 'SB', 'AVG','W','SV','K', 'ERA', 'WHIP'])
    for h in HitterMetrics:
        all_players['norm{}'.format(h)] = norm_players[h]
    for p in PitcherMetrics:
        all_players['norm{}'.format(p)] = norm_players[p]
    
    
    
    all_names = list(all_players.index)
    name_list = list(all_players['Name'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    print(all_players)
    print('------')
    print(all_names)
    
    allRuns = dict(zip(all_names,all_players['normR']))
    allHRs = dict(zip(all_names,all_players['normHR']))
    allRBIs = dict(zip(all_names,all_players['normRBI']))
    allSBs = dict(zip(all_names,all_players['normSB']))
    allAVG = dict(zip(all_names,all_players['normAVG']))

    #all_players['AdjERA'] = all_players['normERA'].apply(lambda x: 0 if x == 0 else 1/x)
    #all_players['AdjWHIP'] = all_players['normWHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    
    allWs = dict(zip(all_names,all_players['normW']))
    allKs = dict(zip(all_names,all_players['normK']))
    allSVs = dict(zip(all_names,all_players['normSV']))
    allERA = dict(zip(all_names,all_players['normERA']*0.9))
    allWHIP = dict(zip(all_names,all_players['normWHIP']*0.9))
    allPOS = dict(zip(all_names,all_players['POS']))

    #allAdjERA = dict(zip(all_names,all_players['AdjERA']))
    #allAdjWHIP = dict(zip(all_names,all_players['AdjWHIP']))
    
    
    player_vars = LpVariable.dicts("Player",all_names,lowBound=0,upBound=1,cat='Integer')
    player_chosen = LpVariable.dicts("Chosen",name_list,0,1,cat='Integer')
    
    print("all plrs: ", len(all_players.index))
    print('vars: ', len(player_vars))
    print('nms: ', len(all_names))
    
    prob += lpSum([allRuns[i]*player_vars[i] for i in all_names])
    prob += lpSum([allHRs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allRBIs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSBs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAVG[i]*player_vars[i] for i in all_names])
    prob += lpSum([allWs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allKs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSVs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allERA[i]*player_vars[i] for i in all_names])
    prob += lpSum([allWHIP[i]*player_vars[i] for i in all_names])
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names])
    
    
#     prob += lpSum([allRuns[i]*player_vars[i] for i in all_names]) <= (hitter_avg_maxes[0]*14)
#     prob += lpSum([allHRs[i]*player_vars[i] for i in all_names]) <= (hitter_avg_maxes[1]*14)
#     prob += lpSum([allRBIs[i]*player_vars[i] for i in all_names]) <= (hitter_avg_maxes[2]*14)
#     prob += lpSum([allSBs[i]*player_vars[i] for i in all_names]) <= (hitter_avg_maxes[3]*14)
#     prob += lpSum([allAVG[i]*player_vars[i] for i in all_names]) <= (hitter_avg_maxes[4])
#     prob += lpSum([allWs[i]*player_vars[i] for i in all_names]) <= (pitcher_avg_maxes[0]*11)
#     prob += lpSum([allKs[i]*player_vars[i] for i in all_names]) <= (pitcher_avg_maxes[1]*11)
#     prob += lpSum([allSVs[i]*player_vars[i] for i in all_names]) <= (pitcher_avg_maxes[2]*11)
#     prob += lpSum([allERA[i]*player_vars[i] for i in all_names]) <= (pitcher_avg_maxes[3]*11)
#     prob += lpSum([allWHIP[i]*player_vars[i] for i in all_names]) <= (pitcher_avg_maxes[4]*11)
    
    
    #prob += lpSum([allRuns[i]*player_vars[i] for i in all_names]) >= (hitter_avg_mins[0]*14)
    #prob += lpSum([allHRs[i]*player_vars[i] for i in all_names]) >= (hitter_avg_mins[1]*14)
    #prob += lpSum([allRBIs[i]*player_vars[i] for i in all_names]) >= (hitter_avg_mins[2]*14)
    #prob += lpSum([allSBs[i]*player_vars[i] for i in all_names]) >= (hitter_avg_mins[3]*14)
    #prob += lpSum([allAVG[i]*player_vars[i] for i in all_names]) >= (hitter_avg_mins[4])
    #prob += lpSum([allWs[i]*player_vars[i] for i in all_names]) >= (pitcher_avg_mins[0]*11)
    #prob += lpSum([allKs[i]*player_vars[i] for i in all_names]) >= (pitcher_avg_mins[1]*11)
    #prob += lpSum([allSVs[i]*player_vars[i] for i in all_names]) >= (pitcher_avg_mins[2]*11)
    #prob += lpSum([allERA[i]*player_vars[i] for i in all_names]) >= (pitcher_avg_mins[3]*11)
    #prob += lpSum([allWHIP[i]*player_vars[i] for i in all_names]) >= (pitcher_avg_mins[4]*11)
    
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 260
    
    prob += lpSum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1
    
    # update these based on position depth
    prob += lpSum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 1
    prob += lpSum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2
    
    prob += lpSum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6
    
    prob += lpSum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8
    
    prob += lpSum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3
    prob += lpSum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0
    
    prob += lpSum([allCosts[i]*player_vars[i]*(allPOS[i]=='RP' or allPOS[i]=='SP') for i in all_names]) <= 120
    prob += lpSum([allCosts[i]*player_vars[i]*(allPOS[i]!='RP' and allPOS[i]!='SP') for i in all_names]) <= 140
    
    # ensure no player is selected twice

    for f in all_names:
        prob += player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.1
        prob += player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e5
    
    prob += lpSum(player_chosen)==25

    LpSolverDefault.msg = 1 
    prob.solve(solver=GUROBI(PoolSearchMode=2,PoolSolutions=4))
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        if v.varValue>0 and not 'Chosen' in v.name:
            Team = Team.append(all_players.loc[int(v.name.split('_')[1])])
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)
 
Team_max = find_max_three()
Team['ERA'] = Team['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
Team['WHIP'] = Team['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
Max_Tot = Team_max[Team_max['Name'] == 'Total']

print('Max',get_score(Max_Tot.loc[25]))
Team_max[['Name','POS','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']]


     AVG rank  SB rank  RBI rank  HR rank  R rank  Razzball_Rank  \
0         7.0     42.0       9.0      1.0     4.0             19   
1        27.0     47.0       1.0      3.0     1.0             31   
2        27.0     47.0       1.0      3.0     1.0             31   
3         3.0     15.0      32.0     24.0     9.0             42   
4        36.0      4.0      36.0     12.0     5.0              6   
..        ...      ...       ...      ...     ...            ...   
295       0.0      0.0       0.0      0.0     0.0            320   
296       0.0      0.0       0.0      0.0     0.0            350   
297       0.0      0.0       0.0      0.0     0.0            187   
298       0.0      0.0       0.0      0.0     0.0            242   
299       0.0      0.0       0.0      0.0     0.0            223   

                 Name Team POS     AB  ...      normR     normHR    normRBI  \
0          Mike Trout  LAA  OF  516.0  ...  71.859333  30.118667  67.583667   
1      Cody Bellinger  LA

/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1380: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


   Value: 2  Min: 0  Max: 2  Default: 0
Parameter PoolSolutions unchanged
   Value: 4  Min: 1  Max: 2000000000  Default: 10
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 613 rows, 575 columns and 2373 nonzeros
Model fingerprint: 0x6a298b1f
Variable types: 0 continuous, 575 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+05]
  Objective range  [1e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 602 rows and 305 columns
Presolve time: 0.00s
Presolved: 11 rows, 270 columns, 788 nonzeros
Variable types: 0 continuous, 270 integer (270 binary)
Found heuristic solution: objective 147.6000000

Root relaxation: objective 2.600000e+02, 46 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  260.00000    0    4  147.60000  260.00000  76.2%     -    0s
H    0 

,Name,POS,R,HR,RBI,SB,AVG,W,SV,K,ERA,WHIP,Ovr,$
0,Andrew McCutchen,OF,80.0,23.3,63.6,6.8,0.259000,0.0,0.0,0.0,0.000000,0.000000,130.40,4.9
1,Kolten Wong,2B,67.9,12.3,62.8,15.8,0.267000,0.0,0.0,0.0,0.000000,0.000000,136.00,5.1
2,Dansby Swanson,SS,74.7,17.6,73.8,10.0,0.253000,0.0,0.0,0.0,0.000000,0.000000,139.00,4.3
3,Anthony Santander,OF,63.9,22.4,75.4,4.8,0.262000,0.0,0.0,0.0,0.000000,0.000000,141.60,2.9
4,Cesar Hernandez,2B,67.8,10.1,54.5,9.1,0.278000,0.0,0.0,0.0,0.000000,0.000000,146.40,1.0
5,Brandon Belt,1B,84.4,20.0,64.9,5.1,0.254000,0.0,0.0,0.0,0.000000,0.000000,153.20,2.6
6,Willy Adames,SS,67.5,17.7,63.1,6.8,0.256000,0.0,0.0,0.0,0.000000,0.000000,159.20,1.0
7,Gerrit Cole,SP,0.0,0.0,0.0,0.0,0.000000,14.9,0.0,273.5,0.615385,1.923077,1.00,35.0
8,Charlie Morton,SP,0.0,0.0,0.0,0.0,0.000000,12.0,0.0,205.9,0.574713,1.680672,11.25,18.6
9,Jack Flaherty,SP,0.0,0.0,0.0,0.0,0.000000,11.6,0.0,214.7,0.557103,1.724138,11.50,19.2


# This is the one \/

In [361]:
def find_gurobi():
    m = Model("mip1")
    # this works really well without using the maxes, but I think it might peform better with them
    
    #all_players = get_all_players()[0]
    x,hitter_avg_maxes,pitcher_avg_maxes,hitter_avg_mins,pitcher_avg_mins = get_all_players()
    scaler = preprocessing.StandardScaler(with_std=False)
    #scaler2 = preprocessing.StandardScaler()
    #scaler2 = preprocessing.MinMaxScaler()
    #all_players,hitter_avg_maxes,pitcher_avg_maxes,hitter_avg_mins,pitcher_avg_mins = get_all_players()
    all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'),ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(22,'SV rank'),ignore_index=True) 
    #all_players.fillna(0, inplace=True)
    all_players = all_players[all_players['Name'] != 'Max Kepler'].reset_index(drop=True)
    all_players.fillna(0, inplace=True)          
    #return(all_players.iloc[290])    
                              
    all_players['ERA'] = all_players['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['WHIP'] = all_players['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AB'] = all_players['AB'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['adjAVG'] = all_players['AVG']*100
    norm_players = all_players[['R', 'HR', 'RBI', 'SB','adjAVG','W','SV','K','ERA','WHIP','AB','H']]
    
    norm_players = pd.DataFrame(scaler.fit_transform(norm_players),columns=['R', 'HR', 'RBI', 'SB','adjAVG','W','SV','K', 'ERA', 'WHIP','H','AB'])
    
#     p_avg_maxes = [(hitter_maxes[i]/14) for i in range(len(hitter_maxes)-1)]
#     p_avg_maxes.append(hitter_maxes[-1]*100)
#     [p_avg_maxes.append(pitcher_maxes[i]/11) for i in range(len(pitcher_maxes)-2)]
#     p_avg_maxes.append(1/pitcher_maxes[-2])
#     p_avg_maxes.append(1/pitcher_maxes[-1])
#     print()
    
#     h_avg_max_ary = np.array(hitter_avg_maxes).reshape(1,-1)
#     p_avg_max_ary = np.array(pitcher_avg_maxes).reshape(1,-1)
#     avg_max_array = np.array(p_avg_maxes).reshape(1,-1)
#     #print(avg_max_array)
#     avg_maxes = scaler.transform(avg_max_array)
#     avg_maxes = avg_maxes[0]
#     print(avg_maxes)
    #hitter_avg_maxes = avg_maxes[0]
    #pitcher_avg_maxes = avg_maxes[1]
    
    
#     maxH = 0
#     minH = math.inf
#     for h in HitterMetrics:
#         if h == 'AVG':
#             continue
#         if(max(norm_players[h])>maxH):
#             maxH = max(norm_players[h])
#         if(min(norm_players[h]<minH)):
#             minH = min(norm_players[h])
#     for h in PitcherMetrics:
#         if(max(norm_players[h])>maxH):
#             maxH = max(norm_players[h])
#         if(min(norm_players[h]<minH)):
#             minH = min(norm_players[h])
            
    #scaler2 = preprocessing.MinMaxScaler((minH,maxH))
    
    #norm_players['AVG'] = all_players[['AVG']]
    
    #norm_players['AVG'] = pd.DataFrame(scaler2.fit_transform(norm_players),columns=['R', 'HR', 'RBI', 'SB','AVG','W','SV','K', 'ERA', 'WHIP'])['AVG']
    for h in HitterMetrics:
        if h == 'AVG':
            all_players['normAVG'] = norm_players['adjAVG']
        else:
            all_players['norm{}'.format(h)] = norm_players[h]
    for p in PitcherMetrics:
        all_players['norm{}'.format(p)] = norm_players[p]
    
    for h in ['AB','H']:
        all_players['norm{}'.format(h)] = norm_players[h]
    
    all_names = list(all_players.index)
    name_list = list(dict.fromkeys(all_players['Name']))
    allCosts = dict(zip(all_names,all_players['$']))
    
    return all_players
    player_vars = m.addVars(all_names,vtype=GRB.INTEGER,lb=0,ub=1,name='players')
    player_chosen = m.addVars(name_list,vtype=GRB.INTEGER,lb=0,ub=1,name='pl_chosen')
    
    allRuns = dict(zip(all_names,all_players['normR']))
    allHRs = dict(zip(all_names,all_players['normHR']))
    allRBIs = dict(zip(all_names,all_players['normRBI']))
    allSBs = dict(zip(all_names,all_players['normSB']*2.5))
    allAVG = dict(zip(all_names,all_players['normAVG']))
    allAB = dict(zip(all_names,all_players['normAB']*0.8))
    allH = dict(zip(all_names,all_players['normH']*0.8))

    #all_players['AdjERA'] = all_players['normERA'].apply(lambda x: 0 if x == 0 else 1/x)
    #all_players['AdjWHIP'] = all_players['normWHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    
    allWs = dict(zip(all_names,all_players['normW']))
    allKs = dict(zip(all_names,all_players['normK']))
    allSVs = dict(zip(all_names,all_players['normSV']))
    allERA = dict(zip(all_names,all_players['normERA']))
    allWHIP = dict(zip(all_names,all_players['normWHIP']))
    allPOS = dict(zip(all_names,all_players['POS']))

    #allAdjERA = dict(zip(all_names,all_players['AdjERA']))
    #allAdjWHIP = dict(zip(all_names,all_players['AdjWHIP']))
    
    
    
    #print("all plrs: ", len(all_players.index))
    ##print('nms: ', len(all_names))
    
    obj = LinExpr()
    
    obj += quicksum([allRuns[i]*player_vars[i] for i in all_names])
    obj += quicksum([allHRs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allRBIs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allSBs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allAVG[i]*player_vars[i] for i in all_names])
    obj += quicksum([allWs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allKs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allSVs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allERA[i]*player_vars[i] for i in all_names])
    obj += quicksum([allWHIP[i]*player_vars[i] for i in all_names])
    obj += quicksum([allCosts[i]*player_vars[i] for i in all_names])
    obj += quicksum([allH[i]*player_vars[i] for i in all_names])
    obj += quicksum([allAB[i]*player_vars[i] for i in all_names])
    m.setObjective(obj, GRB.MAXIMIZE)
    
    m.addConstr(sum([allCosts[i]*player_vars[i] for i in all_names])<= 260)
    
    m.addConstr(sum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1)
    
    # update these based on position depth
    m.addConstr(sum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 2)
    m.addConstr(sum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2)
    m.addConstr(sum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 1)
    m.addConstr(sum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2)
    
    m.addConstr(sum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6)
    
    m.addConstr(sum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8)
    
    m.addConstr(sum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3)
    m.addConstr(sum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0)
    
    
#     m.addConstr(sum([allRuns[i]*player_vars[i] for i in all_names]) <= (avg_maxes[0]*15))
#     m.addConstr(sum([allHRs[i]*player_vars[i] for i in all_names]) <= (avg_maxes[1]*15))
#     m.addConstr(sum([allRBIs[i]*player_vars[i] for i in all_names]) <= (avg_maxes[2]*15))
#     m.addConstr(sum([allSBs[i]*player_vars[i] for i in all_names]) <= (avg_maxes[3]*15))
#     m.addConstr(sum([allAVG[i]*player_vars[i] for i in all_names]) <= (avg_maxes[4]*15))
#     m.addConstr(sum([allWs[i]*player_vars[i] for i in all_names]) <= (avg_maxes[5]*12))
#     m.addConstr(sum([allKs[i]*player_vars[i] for i in all_names]) <= (avg_maxes[6]*12))
#     m.addConstr(sum([allSVs[i]*player_vars[i] for i in all_names]) <= (avg_maxes[7]*12))
#     m.addConstr(sum([allERA[i]*player_vars[i] for i in all_names]) <= (avg_maxes[8]*12))
#     m.addConstr(sum([allWHIP[i]*player_vars[i] for i in all_names]) <= (avg_maxes[9]*12))
    
    
    m.addConstr(sum([allCosts[i]*player_vars[i]*(allPOS[i]=='RP' or allPOS[i]=='SP') for i in all_names]) <= 120)
    m.addConstr(sum([allCosts[i]*player_vars[i]*(allPOS[i]!='RP' and allPOS[i]!='SP') for i in all_names]) <= 140)
    
    
    # ensure no player is selected twice

    for f in all_names:
        m.addConstr(player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.01)
        m.addConstr(player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e8)
    
    
    m.addConstr(sum(player_chosen.values())==25)
    
    m.setParam(GRB.Param.PoolSolutions,10)
    m.setParam(GRB.Param.PoolSearchMode,2)
    
    m.optimize()
    
    nSolns = m.SolCount
    max_tot = 0
    best = 0
    for e in range(nSolns):
        Team = pd.DataFrame()
        m.setParam(GRB.Param.SolutionNumber,e)
        for v in m.getVars():
            if v.Xn > 0.01 and not 'chosen' in v.varName:
                Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
        Team = Team.append(Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True))
        sc = get_score(Team[Team['Name'] == 'Total'].iloc[0])['Total']
        if (sc > max_tot):
            best = e
            max_tot = sc
    print('Best Model: {}'.format(best))
    Team = pd.DataFrame()
    m.setParam(GRB.Param.SolutionNumber,best)
    for v in m.getVars():
        if v.Xn > 0.01 and not 'chosen' in v.varName:
            Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
    
    print('Obj:', m.objVal)
    #Team = pd.DataFrame()
    #print("Status:", LpStatus[prob.status])
#     for v in m.getVars():
#         if v.x > 0.01 and not 'chosen' in v.varName:
#             print(v.varName + ": " + str(v.x))
#             Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
#         if v.x > 0.01 and 'chosen' in v.varName:
#             print(v.varName + ": " + str(v.x))
    
    Team['ERA'] = Team['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    Team['WHIP'] = Team['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)    
Team = find_gurobi()

#Max_Tot = Team[Team['Name'] == 'Total']

#print('Max',get_score(Max_Tot.iloc[0]))
#Team[['Name','POS','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']] 

#all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'),ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(22,'SV rank'),ignore_index=True)

#pl = all_players[all_players['Name'].str.contains('Mike Trout')]
#pl.loc[0,'POS'] == 'SP' or pl.loc[0,'POS'] == 'RP'

#pl = all_players[all_players['Name'].str.contains('mike trout',case=False)].reset_index(drop=True)
#pl2 = pd.DataFrame()
#pl2.count()
Team
#help(tupledict()) 
#Team.columns

/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,AVG rank,SB rank,RBI rank,HR rank,R rank,Razzball_Rank,Name,Team,POS,AB,...,normRBI,normSB,normAVG,normW,normSV,normK,normERA,normWHIP,normAB,normH
0,7.0,41.0,9.0,1.0,4.0,19,Mike Trout,LAA,OF,0.001938,...,66.94433,8.284192,15.717526,-4.02268,-1.993127,-67.464261,-0.121480,-0.389721,77.0,0.000988
1,26.0,47.0,1.0,3.0,1.0,30,Cody Bellinger,LAD,1B,0.001799,...,75.64433,7.584192,14.717526,-4.02268,-1.993127,-67.464261,-0.121480,-0.389721,83.4,0.000849
2,26.0,47.0,1.0,3.0,1.0,30,Cody Bellinger,LAD,OF,0.001799,...,75.64433,7.584192,14.717526,-4.02268,-1.993127,-67.464261,-0.121480,-0.389721,83.4,0.000849
3,3.0,15.0,32.0,25.0,9.0,41,Christian Yelich,MIL,OF,0.001815,...,53.94433,16.484192,16.417526,-4.02268,-1.993127,-67.464261,-0.121480,-0.389721,91.4,0.000865
4,40.0,4.0,36.0,12.0,5.0,6,Ronald Acuna Jr.,ATL,OF,0.001695,...,52.24433,24.484192,14.317526,-4.02268,-1.993127,-67.464261,-0.121480,-0.389721,90.7,0.000745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,0.0,0.0,0.0,0.0,0.0,281,Joe Jimenez,DET,RP,0.000000,...,-42.55567,-4.915808,-13.982474,-1.22268,18.006873,1.635739,0.119483,0.403930,-76.1,-0.000950
287,0.0,0.0,0.0,0.0,0.0,242,Ian Kennedy,KC,RP,0.000000,...,-42.55567,-4.915808,-13.982474,-0.92268,18.006873,-0.564261,0.110001,0.410279,-76.1,-0.000950
288,0.0,0.0,0.0,0.0,0.0,295,Mark Melancon,ATL,RP,0.000000,...,-42.55567,-4.915808,-13.982474,-1.02268,18.006873,-13.864261,0.149522,0.391529,-76.1,-0.000950
289,0.0,0.0,0.0,0.0,0.0,215,Hansel Robles,LAA,RP,0.000000,...,-42.55567,-4.915808,-13.982474,-0.72268,18.006873,3.435739,0.117754,0.397681,-76.1,-0.000950


In [212]:
print(get_pitcher_prices(get_closing_pitchers()).nsmallest(30,'SV rank')[['Name','$','SV']])

                 Name     $  SV
10      Roberto Osuna  16.3  35
5     Aroldis Chapman  15.7  35
2         Kirby Yates  18.9  35
18      Kenley Jansen  13.3  30
12          Brad Hand  13.3  30
20    Raisel Iglesias  13.6  30
8       Taylor Rogers  14.2  30
0          Josh Hader  23.0  30
40    Brandon Workman   7.4  25
1          Edwin Diaz  17.4  25
62        Alex Colome   6.0  25
14          Ken Giles  10.4  25
11      Craig Kimbrel  10.5  25
4       Liam Hendriks  13.1  25
22       Hector Neris   9.2  25
32     Sean Doolittle   8.1  25
43     Archie Bradley   6.3  25
39        Joe Jimenez   4.7  20
36      Mark Melancon   3.8  20
51        Ian Kennedy   4.7  20
9        Jose Leclerc  10.7  20
42      Hansel Robles   5.4  20
3       Nick Anderson  10.8  15
15         Keone Kela   6.7  15
34        Tony Watson   1.0  10
69   Brandon Kintzler   1.0  10
276        Wade Davis   1.0  10
28      Mychal Givens   3.7  10
6          Will Smith   5.9   8
94        Scott Oberg   1.0   7


In [239]:
def find_newnorm():
    prob = LpProblem("IdealRosterProblem",LpMaximize)
    
    
    all_players = get_all_players()
    
    
    all_names = list(all_players.index)
    name_list = list(all_players['Name'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    
    allRuns = dict(zip(all_names,all_players['R']/max(hitter_scores['R'])))
    allHRs = dict(zip(all_names,all_players['HR']/max(hitter_scores['HR'])))
    allRBIs = dict(zip(all_names,all_players['RBI']/max(hitter_scores['RBI'])))
    allSBs = dict(zip(all_names,all_players['SB']/max(hitter_scores['SB'])))
    allAVG = dict(zip(all_names,all_players['AVG']/max(hitter_scores['AVG'])))

    all_players['AdjERA'] = all_players['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjWHIP'] = all_players['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    
    totAdjERA = 1/min(pitcher_scores['ERA'])
    totAdjWHIP = 1/min(pitcher_scores['WHIP'])
    
    allWs = dict(zip(all_names,all_players['W']/max(pitcher_scores['W'])))
    allKs = dict(zip(all_names,all_players['K']/max(pitcher_scores['K'])))
    allSVs = dict(zip(all_names,all_players['SV']/max(pitcher_scores['SV'])))
    allERA = dict(zip(all_names,all_players['AdjERA']/totAdjERA))
    allWHIP = dict(zip(all_names,all_players['AdjWHIP']/totAdjWHIP))
    allPOS = dict(zip(all_names,all_players['POS']))

    allAdjERA = dict(zip(all_names,all_players['AdjERA']))
    allAdjWHIP = dict(zip(all_names,all_players['AdjWHIP']))
    
    
    player_vars = LpVariable.dicts("Player",all_names,lowBound=0,upBound=1,cat='Integer')
    player_chosen = LpVariable.dicts("Chosen",name_list,0,1,cat='Integer')
    
    print("all plrs: ", len(all_players.index))
    print('vars: ', len(player_vars))
    print('nms: ', len(all_names))
    
    prob += lpSum([allRuns[i]*player_vars[i] for i in all_names])
    prob += lpSum([allHRs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allRBIs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSBs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAVG[i]*player_vars[i] for i in all_names])
    prob += lpSum([allWs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allKs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSVs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAdjERA[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAdjWHIP[i]*player_vars[i] for i in all_names])
    
    
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 260
    
    prob += lpSum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1
    
    # update these based on position depth
    prob += lpSum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 1
    prob += lpSum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2
    
    prob += lpSum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6
    
    prob += lpSum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8
    
    prob += lpSum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3
    prob += lpSum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0
    
    
    # ensure no player is selected twice

    for f in all_names:
        prob += player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.1
        prob += player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e5
    
    prob += lpSum(player_chosen)==25

    LpSolverDefault.msg = 1 
    prob.solve(solver=GUROBI())
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        if v.varValue>0 and not 'Chosen' in v.name:
            Team = Team.append(all_players.loc[int(v.name.split('_')[1])])
            
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)
 
Team_max = find_newnorm()
Max_Tot = Team_max[Team_max['Name'] == 'Total']

print('Max',get_score(Max_Tot.loc[25]))
Team_max[['Name','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']]


/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
def find_max_off():
    prob = LpProblem("IdealRosterProblem",LpMaximize)
    
    scaler = preprocessing.MinMaxScaler()
    
    hitters = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr')
    norm_hitters = hitters[['R', 'HR', 'RBI', 'SB', 'AVG']]
    norm_hitters = pd.DataFrame(scaler.fit_transform(norm_hitters),columns=['R', 'HR', 'RBI', 'SB', 'AVG'])
    for h in HitterMetrics:
        hitters['norm{}'.format(h)] = norm_hitters[h]
    
    all_players = hitters
    
    
    all_names = list(all_players.index)
    name_list = list(all_players['Name'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    
    allRuns = dict(zip(all_names,all_players['normR']))
    allHRs = dict(zip(all_names,all_players['normHR']))
    allRBIs = dict(zip(all_names,all_players['normRBI']))
    allSBs = dict(zip(all_names,all_players['normSB']))
    allAVG = dict(zip(all_names,all_players['normAVG']))
    allPOS = dict(zip(all_names,all_players['POS']))
    
    
    player_vars = LpVariable.dicts("Player",all_names,lowBound=0,upBound=1,cat='Integer')
    player_chosen = LpVariable.dicts("Chosen",name_list,0,1,cat='Integer')
    
    print("all plrs: ", len(all_players.index))
    print('vars: ', len(player_vars))
    print('nms: ', len(all_names))
    
    prob += lpSum([allRuns[i]*player_vars[i] for i in all_names])
    prob += lpSum([allHRs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allRBIs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSBs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAVG[i]*player_vars[i] for i in all_names])
    
    
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 155
    
    prob += lpSum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1
    
    # update these based on position depth
    prob += lpSum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 1
    prob += lpSum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2
    
    prob += lpSum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6
    
    prob += lpSum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0
    
    
    # ensure no player is selected twice

    for f in all_names:
        prob += player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.1
        prob += player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e5
    
    prob += lpSum(player_chosen)==14

    LpSolverDefault.msg = 1 
    prob.solve()
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        if v.varValue>0 and not 'Chosen' in v.name:
            Team = Team.append(all_players.loc[int(v.name.split('_')[1])])
            
    return Team#Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'$':sum(Team['$'])},ignore_index=True)
 
def find_max_def():
    scaler = preprocessing.MinMaxScaler()
    
    starters = get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr')
    norm_starters = starters[['W']]
    norm_starters = pd.DataFrame(scaler.fit_transform(norm_starters),columns=['W'])
    starters['normW'] = norm_starters['W']
        
    closers = get_pitcher_prices(get_closing_pitchers()).nsmallest(30,'SV rank')
    norm_closers = closers[['SV']]
    norm_closers = pd.DataFrame(scaler.fit_transform(norm_closers),columns=['SV'])
    closers['normSV'] = norm_closers['SV']
        
    all_pitchers = starters.append(closers, ignore_index=True)
    norm_pitchers = all_pitchers[['K', 'ERA', 'WHIP']]
    norm_pitchers = pd.DataFrame(scaler.fit_transform(norm_pitchers),columns=['K', 'ERA', 'WHIP'])
    for p in ['K', 'ERA', 'WHIP']:
        all_pitchers['norm{}'.format(p)] = norm_pitchers[p]
        
    prob = LpProblem("IdealRosterProblem",LpMaximize)
    
    
    all_players = all_pitchers
    
    
    all_names = list(all_players.index)
    name_list = list(all_players['Name'])
    allCosts = dict(zip(all_names,all_players['$']))

    all_players['AdjERA'] = all_players['normERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AdjWHIP'] = all_players['normWHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    
    allWs = dict(zip(all_names,all_players['normW']))
    allKs = dict(zip(all_names,all_players['normK']))
    allSVs = dict(zip(all_names,all_players['normSV']))
    allERA = dict(zip(all_names,all_players['normERA']))
    allWHIP = dict(zip(all_names,all_players['normWHIP']))
    allPOS = dict(zip(all_names,all_players['POS']))

    allAdjERA = dict(zip(all_names,all_players['AdjERA']))
    allAdjWHIP = dict(zip(all_names,all_players['AdjWHIP']))
    
    
    player_vars = LpVariable.dicts("Player",all_names,lowBound=0,upBound=1,cat='Integer')
    player_chosen = LpVariable.dicts("Chosen",name_list,0,1,cat='Integer')
    
    print("all plrs: ", len(all_players.index))
    print('vars: ', len(player_vars))
    print('nms: ', len(all_names))
    
    prob += lpSum([allWs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allKs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allSVs[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAdjERA[i]*player_vars[i] for i in all_names])
    prob += lpSum([allAdjWHIP[i]*player_vars[i] for i in all_names])
    
    
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 105
    
    
    prob += lpSum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8
    
    prob += lpSum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3
    prob += lpSum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0
    
    
    # ensure no player is selected twice

    for f in all_names:
        prob += player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.1
        prob += player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e5
    
    prob += lpSum(player_chosen)==11

    LpSolverDefault.msg = 1 
    prob.solve()
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        if v.varValue>0 and not 'Chosen' in v.name:
            Team = Team.append(all_players.loc[int(v.name.split('_')[1])])
            
    return Team#Team.append({'Name':'Total','WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)
 
Team_max_off = find_max_off()
Team_max_def = find_max_def()

Team_max = Team_max_off.append(Team_max_def, ignore_index=True)
Team_max
Team_max.fillna(0, inplace=True)
Team_max = Team_max.append({'Name':'Total','AVG':sum(Team_max['H'])/sum(Team_max['AB']),'R':sum(Team_max['R']),'RBI':sum(Team_max['RBI']),'HR':sum(Team_max['HR']),'SB':sum(Team_max['SB']),'WHIP':(sum(Team_max['BB'])+sum(Team_max['Hits']))/sum(Team_max['IP']),'ERA':(sum(Team_max['ER'])/sum(Team_max['IP']))*9,'W':sum(Team_max['W']),'SV':sum(Team_max['SV']),'K':sum(Team_max['K']),'$':sum(Team_max['$'])},ignore_index=True)
Team_max[['Name','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']]
Max_Tot = Team_max[Team_max['Name'] == 'Total']
Max_Tot[['Name','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']]
#print(Max_Tot)
print('Max',get_score(Max_Tot.loc[25]))
Team_max[['Name','POS','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']]


In [13]:
GUROBI().available()

True

In [32]:
get_all_players()[['normW','normSV','normK','normERA','normWHIP','normR','normHR','normRBI','normSB','normAVG']].describe()

/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,normW,normSV,normK,normERA,normWHIP,normR,normHR,normRBI,normSB,normAVG
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.208201,0.040119,0.200778,0.272091,0.247971,0.232018,0.234332,0.197644,0.094512,0.239333
std,0.287000,0.169822,0.254647,0.307216,0.281845,0.292863,0.289443,0.250578,0.159518,0.284130
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.468254,0.000000,0.401035,0.570455,0.543478,0.475539,0.430836,0.357531,0.119369,0.476923
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [314]:
def find_w_selected(budget,selected):
    m = Model("mip1")
    # this works really well without using the maxes, but I think it might peform better with them
    
    x,hitter_avg_maxes,pitcher_avg_maxes,hitter_avg_mins,pitcher_avg_mins = get_all_players()
    scaler = preprocessing.StandardScaler(with_std=False)
    all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'),ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(22,'SV rank'),ignore_index=True)
    all_players.fillna(0, inplace=True)
    all_players['ERA'] = all_players['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['WHIP'] = all_players['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AB'] = all_players['AB'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['adjAVG'] = all_players['AVG']*100
    norm_players = all_players[['R', 'HR', 'RBI', 'SB','adjAVG','W','SV','K','ERA','WHIP','AB','H']]
    
    norm_players = pd.DataFrame(scaler.fit_transform(norm_players),columns=['R', 'HR', 'RBI', 'SB','adjAVG','W','SV','K', 'ERA', 'WHIP','H','AB'])
    
    for h in HitterMetrics:
        if h == 'AVG':
            all_players['normAVG'] = norm_players['adjAVG']
        else:
            all_players['norm{}'.format(h)] = norm_players[h]
    for p in PitcherMetrics:
        all_players['norm{}'.format(p)] = norm_players[p]
    
    for h in ['AB','H']:
        all_players['norm{}'.format(h)] = norm_players[h]
    
    all_names = list(all_players.index)
    name_list = list(dict.fromkeys(all_players['Name']))
    for s in selected:
        all_players.loc[all_players['Name'] == s,'$'] = 0
        print(all_players[all_players['Name'] == s]['$'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    player_vars = m.addVars(all_names,vtype=GRB.INTEGER,lb=0,ub=1,name='players')
    player_chosen = m.addVars(name_list,vtype=GRB.INTEGER,lb=0,ub=1,name='pl_chosen')
    
    allRuns = dict(zip(all_names,all_players['normR']))
    allHRs = dict(zip(all_names,all_players['normHR']))
    allRBIs = dict(zip(all_names,all_players['normRBI']))
    allSBs = dict(zip(all_names,all_players['normSB']*2.5))
    allAVG = dict(zip(all_names,all_players['normAVG']))
    allAB = dict(zip(all_names,all_players['normAB']*0.8))
    allH = dict(zip(all_names,all_players['normH']*0.8))

    
    allWs = dict(zip(all_names,all_players['normW']))
    allKs = dict(zip(all_names,all_players['normK']))
    allSVs = dict(zip(all_names,all_players['normSV']))
    allERA = dict(zip(all_names,all_players['normERA']))
    allWHIP = dict(zip(all_names,all_players['normWHIP']))
    allPOS = dict(zip(all_names,all_players['POS']))
    
    obj = LinExpr()
    
    obj += quicksum([allRuns[i]*player_vars[i] for i in all_names])
    obj += quicksum([allHRs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allRBIs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allSBs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allAVG[i]*player_vars[i] for i in all_names])
    obj += quicksum([allWs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allKs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allSVs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allERA[i]*player_vars[i] for i in all_names])
    obj += quicksum([allWHIP[i]*player_vars[i] for i in all_names])
    obj += quicksum([allCosts[i]*player_vars[i] for i in all_names])
    obj += quicksum([allH[i]*player_vars[i] for i in all_names])
    obj += quicksum([allAB[i]*player_vars[i] for i in all_names])
    m.setObjective(obj, GRB.MAXIMIZE)
    
    m.addConstr(sum([allCosts[i]*player_vars[i] for i in all_names])<= budget)
    
    m.addConstr(sum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1)
    
    # update these based on position depth
    m.addConstr(sum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 2)
    m.addConstr(sum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2)
    m.addConstr(sum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 1)
    m.addConstr(sum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2)
    
    m.addConstr(sum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6)
    
    m.addConstr(sum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8)
    
    m.addConstr(sum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3)
    m.addConstr(sum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0)
    
    
    m.addConstr(sum([allCosts[i]*player_vars[i]*(allPOS[i]=='RP' or allPOS[i]=='SP') for i in all_names]) <= 120)
    m.addConstr(sum([allCosts[i]*player_vars[i]*(allPOS[i]!='RP' and allPOS[i]!='SP') for i in all_names]) <= 140)
    
    
    # ensure no player is selected twice

    for f in all_names:
        m.addConstr(player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.01)
        m.addConstr(player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e8)
    
    
    m.addConstr(sum(player_chosen.values())==25)
    
    for s in selected:
        m.addConstr(player_chosen[s] >= 0.7)
    
    m.setParam(GRB.Param.PoolSolutions,10)
    m.setParam(GRB.Param.PoolSearchMode,2)
    
    m.optimize()
    
    nSolns = m.SolCount
    max_tot = 0
    best = 0
    for e in range(nSolns):
        Team = pd.DataFrame()
        m.setParam(GRB.Param.SolutionNumber,e)
        for v in m.getVars():
            if v.Xn > 0.01 and not 'chosen' in v.varName:
                Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
        Team = Team.append(Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True))
        sc = get_score(Team[Team['Name'] == 'Total'].iloc[0])['Total']
        if (sc > max_tot):
            best = e
            max_tot = sc
    print('Best Model: {}'.format(best))
    Team = pd.DataFrame()
    m.setParam(GRB.Param.SolutionNumber,best)
    for v in m.getVars():
        if v.Xn > 0.01 and not 'chosen' in v.varName:
            Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
    
    print('Obj:', m.objVal)
    
    Team['ERA'] = Team['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    Team['WHIP'] = Team['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)    
Team = find_w_selected(220,['Taylor Rogers',"J.T. Realmuto"])

Max_Tot = Team[Team['Name'] == 'Total']

print('Max',get_score(Max_Tot.iloc[0]))
Team[['Name','POS','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']] 
#help(tupledict()) 
    

/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/samozminkowski/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


276    0.0
Name: $, dtype: float64
81    0.0
Name: $, dtype: float64
Changed value of parameter PoolSolutions to 10
   Prev: 4  Min: 1  Max: 2000000000  Default: 10
Parameter PoolSearchMode unchanged
   Value: 2  Min: 0  Max: 2  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 599 rows, 559 columns and 2307 nonzeros
Model fingerprint: 0xc87bb543
Variable types: 0 continuous, 559 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+08]
  Objective range  [5e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-01, 2e+02]
Presolve removed 588 rows and 298 columns
Presolve time: 0.00s
Presolved: 11 rows, 261 columns, 767 nonzeros
Variable types: 0 continuous, 261 integer (261 binary)

Root relaxation: objective 5.432012e+02, 32 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  

,Name,POS,R,HR,RBI,SB,AVG,W,SV,K,ERA,WHIP,Ovr,$
0,Ozzie Albies,2B,99.6,24.4,86.9,13.9,0.288000,0.0,0.0,0.0,0.000000,0.000000,44.60,23.5
1,Keston Hiura,2B,94.0,30.6,91.8,13.5,0.271000,0.0,0.0,0.0,0.000000,0.000000,56.20,21.7
2,Andrew Benintendi,OF,99.3,19.2,78.1,13.5,0.272000,0.0,0.0,0.0,0.000000,0.000000,77.80,16.3
3,Max Kepler,OF,103.7,31.1,88.4,6.0,0.259000,0.0,0.0,0.0,0.000000,0.000000,91.20,16.1
4,J.T. Realmuto,C,77.8,23.9,76.4,5.8,0.270000,0.0,0.0,0.0,0.000000,0.000000,108.00,0.0
5,Matt Chapman,3B,94.1,34.7,98.1,2.7,0.258000,0.0,0.0,0.0,0.000000,0.000000,114.40,15.0
6,Eric Hosmer,1B,79.6,23.4,80.2,3.4,0.263000,0.0,0.0,0.0,0.000000,0.000000,128.80,6.6
7,Elvis Andrus,SS,69.3,13.5,66.0,20.8,0.262000,0.0,0.0,0.0,0.000000,0.000000,133.80,8.4
8,Oscar Mercado,OF,75.2,15.7,65.4,23.4,0.256000,0.0,0.0,0.0,0.000000,0.000000,134.40,10.6
9,Dansby Swanson,SS,74.7,17.6,73.8,10.0,0.253000,0.0,0.0,0.0,0.000000,0.000000,139.00,4.3


In [2]:
def simple_gurobi():
    m = Model("mip1")
    
    x,hitter_avg_maxes,pitcher_avg_maxes,hitter_avg_mins,pitcher_avg_mins = get_all_players()
    scaler = preprocessing.StandardScaler(with_std=False)
    
    all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'),ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(22,'SV rank'),ignore_index=True) 
    
    all_players = all_players[all_players['Name'] != 'Max Kepler'].reset_index(drop=True)
    all_players.fillna(0, inplace=True)          
        
                              
    all_players['ERA'] = all_players['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['WHIP'] = all_players['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['AB'] = all_players['AB'].apply(lambda x: 0 if x == 0 else 1/x)
    all_players['adjAVG'] = all_players['AVG']*100
    norm_players = all_players[['R', 'HR', 'RBI', 'SB','adjAVG','W','SV','K','ERA','WHIP','AB','H']]
    
    norm_players = pd.DataFrame(scaler.fit_transform(norm_players),columns=['R', 'HR', 'RBI', 'SB','adjAVG','W','SV','K', 'ERA', 'WHIP','H','AB'])
    
    for h in HitterMetrics:
        if h == 'AVG':
            all_players['normAVG'] = norm_players['adjAVG']
        else:
            all_players['norm{}'.format(h)] = norm_players[h]
    for p in PitcherMetrics:
        all_players['norm{}'.format(p)] = norm_players[p]
    
    for h in ['AB','H']:
        all_players['norm{}'.format(h)] = norm_players[h]
    
    all_names = list(all_players.index)
    name_list = list(dict.fromkeys(all_players['Name']))
    allCosts = dict(zip(all_names,all_players['$']))
    
    player_vars = m.addVars(all_names,vtype=GRB.INTEGER,lb=0,ub=1,name='players')
    player_chosen = m.addVars(name_list,vtype=GRB.INTEGER,lb=0,ub=1,name='pl_chosen')
    
    allRuns = dict(zip(all_names,all_players['normR']))
    allHRs = dict(zip(all_names,all_players['normHR']))
    allRBIs = dict(zip(all_names,all_players['normRBI']))
    allSBs = dict(zip(all_names,all_players['normSB']))
    allAVG = dict(zip(all_names,all_players['normAVG']))
    allAB = dict(zip(all_names,all_players['normAB']))
    allH = dict(zip(all_names,all_players['normH']))

    
    allWs = dict(zip(all_names,all_players['normW']))
    allKs = dict(zip(all_names,all_players['normK']))
    allSVs = dict(zip(all_names,all_players['normSV']))
    allERA = dict(zip(all_names,all_players['normERA']))
    allWHIP = dict(zip(all_names,all_players['normWHIP']))
    allPOS = dict(zip(all_names,all_players['POS']))

    
    obj = LinExpr()
    
    obj += quicksum([allRuns[i]*player_vars[i] for i in all_names])
    obj += quicksum([allHRs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allRBIs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allSBs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allAVG[i]*player_vars[i] for i in all_names])
    obj += quicksum([allWs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allKs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allSVs[i]*player_vars[i] for i in all_names])
    obj += quicksum([allERA[i]*player_vars[i] for i in all_names])
    obj += quicksum([allWHIP[i]*player_vars[i] for i in all_names])
    obj += quicksum([allCosts[i]*player_vars[i] for i in all_names])
    obj += quicksum([allH[i]*player_vars[i] for i in all_names])
    obj += quicksum([allAB[i]*player_vars[i] for i in all_names])
    m.setObjective(obj, GRB.MAXIMIZE)
    
    m.addConstr(sum([allCosts[i]*player_vars[i] for i in all_names])<= 260)
    
    m.addConstr(sum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1)
    
    # update these based on position depth
    m.addConstr(sum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 2)
    m.addConstr(sum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2)
    m.addConstr(sum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 1)
    m.addConstr(sum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2)
    
    m.addConstr(sum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6)
    
    m.addConstr(sum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8)
    
    m.addConstr(sum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3)
    m.addConstr(sum([(allPOS[i]=='DH')*player_vars[i] for i in all_names]) == 0)
    
    
    m.addConstr(sum([allCosts[i]*player_vars[i]*(allPOS[i]=='RP' or allPOS[i]=='SP') for i in all_names]) <= 120)
    m.addConstr(sum([allCosts[i]*player_vars[i]*(allPOS[i]!='RP' and allPOS[i]!='SP') for i in all_names]) <= 140)
    
    
    # ensure no player is selected twice

    for f in all_names:
        m.addConstr(player_vars[f]>= player_chosen[all_players.iloc[f]['Name']]*0.01)
        m.addConstr(player_vars[f]<= player_chosen[all_players.iloc[f]['Name']]*1e8)
    
    
    m.addConstr(sum(player_chosen.values())==25)
    
    m.setParam(GRB.Param.PoolSolutions,10)
    m.setParam(GRB.Param.PoolSearchMode,2)
    
    m.optimize()
    
#    for v in m.getVars():
#        if v.Xn > 0.01 and not 'chosen' in v.varName:
#            Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
#    Team = Team.append(Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True))

#    Team = pd.DataFrame()
#    for v in m.getVars():
#        if v.Xn > 0.01 and not 'chosen' in v.varName:
#            Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
    
#    print('Obj:', m.objVal)
    Team = pd.DataFrame()
    print("Status:", LpStatus[prob.status])
    for v in m.getVars():
        if v.x > 0.01 and not 'chosen' in v.varName:
            print(v.varName + ": " + str(v.x))
            Team = Team.append(all_players.iloc[int(v.varName.split('[')[1].split(']')[0])])
        if v.x > 0.01 and 'chosen' in v.varName:
            print(v.varName + ": " + str(v.x))
    
    Team['ERA'] = Team['ERA'].apply(lambda x: 0 if x == 0 else 1/x)
    Team['WHIP'] = Team['WHIP'].apply(lambda x: 0 if x == 0 else 1/x)
    return Team.append({'Name':'Total','AVG':sum(Team['H'])/sum(Team['AB']),'R':sum(Team['R']),'RBI':sum(Team['RBI']),'HR':sum(Team['HR']),'SB':sum(Team['SB']),'WHIP':(sum(Team['BB'])+sum(Team['Hits']))/sum(Team['IP']),'ERA':(sum(Team['ER'])/sum(Team['IP']))*9,'W':sum(Team['W']),'SV':sum(Team['SV']),'K':sum(Team['K']),'$':sum(Team['$'])},ignore_index=True)    
Team = simple_gurobi()

#Max_Tot = Team[Team['Name'] == 'Total']

#print('Max',get_score(Max_Tot.iloc[0]))
#Team[['Name','POS','R', 'HR', 'RBI', 'SB', 'AVG','W', 'SV', 'K', 'ERA', 'WHIP','Ovr','$']] 

#all_players = get_hitter_prices(get_hitters()).nsmallest(150,'Ovr').append(get_pitcher_prices(get_starting_pitchers()).nsmallest(120,'Ovr'),ignore_index=True).append(get_pitcher_prices(get_closing_pitchers()).nsmallest(22,'SV rank'),ignore_index=True)

#pl = all_players[all_players['Name'].str.contains('Mike Trout')]
#pl.loc[0,'POS'] == 'SP' or pl.loc[0,'POS'] == 'RP'

#pl = all_players[all_players['Name'].str.contains('mike trout',case=False)].reset_index(drop=True)
#pl2 = pd.DataFrame()
#pl2.count()
Team
#help(tupledict()) 
#Team.columns

NameError: name 'Model' is not defined